<a href="https://colab.research.google.com/github/LeahDiskin/DataProject/blob/main/other-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will demonstrate the work on the CIFAR-10 data set from the simplest Softmax model (Logistic regression) to the more advanced methods

## 1. Introduction
The CIFAR-10 dataset contains 60,000 color images of 32 x 32 pixels in 3 channels divided into 10
classes. Each class contains 6,000 images. The training set contains 50,000 images, while the test sets
provides 10,000 images. This image taken from the CIFAR repository ( <a href = "https://www.cs.toronto.edu/~kriz/cifar.html">https://www.cs.toronto.edu/~kriz/cifar.html </a>). This is a classification problem with 10 classes(muti-label classification). We can take a view on this image for more comprehension of the dataset. 

![cifar10.png](https://github.com/borislevant/SciComPy/blob/master/cifar10.png?raw=1)


The challenge is to recognize previously unseen images and assign them to one of the 10 classes.

Ok Let's get started.

## 2. Import and Preprocess the data



### 2.1 Import all required libraries

In [1]:
from __future__ import print_function
from tensorflow import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, InputLayer
from keras.layers import Conv2D, MaxPooling2D
import os

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report
import itertools
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

%matplotlib inline


In [2]:
batch_size = 32  # The default batch size of keras.
num_classes = 15  # Number of class for the dataset
epochs = 2000
data_augmentation = False

opt = keras.optimizers.Adam(learning_rate=0.0001)
reg = keras.regularizers.l2(l2=0.0001)

In [3]:
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
data=np.load(r"drive/MyDrive/DataProject/data.npz")

Mounted at /content/drive


In [4]:
x_train=data['x_train']
y_train=data['y_train']
x_validation=data['x_validation']
y_validation=data['y_validation']
x_test=data['x_test']
y_test=data['y_test']
y_train = np.reshape(y_train, (-1, 1))
y_validation = np.reshape(y_validation, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))

### 2.2 Import and preproces of data 
We load the data and split it between train and test sets


In [ ]:
# The data, split between train and test sets:
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# print('x_train shape:', x_train.shape)
# print('y_train shape:', y_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')


170508288/170498071 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_validation=x_validation.astype('float32')
x_train /= 255
x_test /= 255
x_validation/=255

# Convert class vectors to binary class matrices. This is called one hot encoding.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_validation = keras.utils.to_categorical(y_validation, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(x_validation.shape)
print(y_validation.shape)

(15000, 32, 32, 3)
(15000, 15)


##3. Softmax model

In [6]:
model = Sequential()
model.add(InputLayer(input_shape=x_train.shape[1:]))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax', kernel_regularizer=reg))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 15)                46095     
                                                                 
Total params: 46,095
Trainable params: 46,095
Non-trainable params: 0
_________________________________________________________________


In [43]:

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
# fit model
# history = model.fit(trainX, trainy, validation_data=(testX, testy), epochs=4000, verbose=0, callbacks=[es, mc])
history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_validation, y_validation),
              shuffle=True, callbacks=[es, mc])

# model.save(r'/content/drive/MyDrive/projects/model_softmax', save_format='h5')
# pd.DataFrame(history.history).to_csv('/content/drive/MyDrive/projects/history_softmax.csv')


Epoch 1/2000
1406/1407 [============================>.] - ETA: 0s - loss: 2.3548 - accuracy: 0.2387
Epoch 1: val_accuracy improved from -inf to 0.29280, saving model to best_model.h5
1407/1407 [==============================] - 5s 4ms/step - loss: 2.3548 - accuracy: 0.2387 - val_loss: 2.2179 - val_accuracy: 0.2928
Epoch 2/2000
1407/1407 [==============================] - ETA: 0s - loss: 2.1955 - accuracy: 0.2981
Epoch 2: val_accuracy improved from 0.29280 to 0.30493, saving model to best_model.h5
1407/1407 [==============================] - 5s 3ms/step - loss: 2.1955 - accuracy: 0.2981 - val_loss: 2.1680 - val_accuracy: 0.3049
Epoch 3/2000
1407/1407 [==============================] - ETA: 0s - loss: 2.1457 - accuracy: 0.3170
Epoch 3: val_accuracy improved from 0.30493 to 0.31800, saving model to best_model.h5
1407/1407 [==============================] - 5s 3ms/step - loss: 2.1457 - accuracy: 0.3170 - val_loss: 2.1265 - val_accuracy: 0.3180
Epoch 4/2000
1396/1407 [======================

In [46]:
model.save(r'/content/drive/MyDrive/projects/model_softmax', save_format='h5')
pd.DataFrame(history.history).to_csv('/content/drive/MyDrive/projects/history_softmax.csv')

In [34]:
# from keras.callbacks import History, EarlyStopping

# history = History()
# callback = [history, EarlyStopping(monitor='val_loss', patience=5, verbose=1, min_delta=1e-4)]

# history = model.fit_generator(x_train, y_train,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_data=(x_validation, y_validation),
#               shuffle=True, 
#               callbacks=callbacks)
# number_of_epochs_it_ran = len(history.history['loss'])

NameError: ignored

##4. Fully-connected Neural Network (FNN) model

In [47]:
model = Sequential()
model.add(InputLayer(input_shape=x_train.shape[1:]))
model.add(Flatten())
model.add(Dense(1024, activation='relu', kernel_regularizer=reg))
model.add(Dense(256, activation='relu', kernel_regularizer=reg))
model.add(Dense(64, activation='relu', kernel_regularizer=reg))
model.add(Dense(num_classes, activation='softmax', kernel_regularizer=reg))
model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 3072)              0         
                                                                 
 dense_23 (Dense)            (None, 1024)              3146752   
                                                                 
 dense_24 (Dense)            (None, 256)               262400    
                                                                 
 dense_25 (Dense)            (None, 64)                16448     
                                                                 
 dense_26 (Dense)            (None, 15)                975       
                                                                 
Total params: 3,426,575
Trainable params: 3,426,575
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_validation, y_validation),
              shuffle=True, callbacks=[es, mc])

model.save(r'/content/drive/MyDrive/DataProject/model_fnn', save_format='h5')
pd.DataFrame(history.history).to_csv('/content/drive/MyDrive/DataProject/history_fnn.csv')


Epoch 1/2000
1407/1407 [==============================] - ETA: 0s - loss: 2.4226 - accuracy: 0.2527
Epoch 1: val_accuracy improved from -inf to 0.30093, saving model to best_model.h5
1407/1407 [==============================] - 7s 5ms/step - loss: 2.4226 - accuracy: 0.2527 - val_loss: 2.2266 - val_accuracy: 0.3009
Epoch 2/2000
1404/1407 [============================>.] - ETA: 0s - loss: 2.1500 - accuracy: 0.3236
Epoch 2: val_accuracy improved from 0.30093 to 0.33553, saving model to best_model.h5
1407/1407 [==============================] - 7s 5ms/step - loss: 2.1498 - accuracy: 0.3235 - val_loss: 2.0848 - val_accuracy: 0.3355
Epoch 3/2000
1404/1407 [============================>.] - ETA: 0s - loss: 2.0351 - accuracy: 0.3555
Epoch 3: val_accuracy improved from 0.33553 to 0.36607, saving model to best_model.h5
1407/1407 [==============================] - 7s 5ms/step - loss: 2.0349 - accuracy: 0.3556 - val_loss: 1.9964 - val_accuracy: 0.3661
Epoch 4/2000
1401/1407 [======================

# 5. Convolutional Neural Network

In [15]:
#define the convnet
model = Sequential()
# CONV => RELU => CONV => RELU => POOL => DROPOUT
model.add(Conv2D(32, (3, 3), padding='same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# CONV => RELU => CONV => RELU => POOL => DROPOUT
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# FLATTERN => DENSE => RELU => DROPOUT
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# a softmax classifier
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 activation_6 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_7 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 15, 15, 32)        0         
                                                      

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_validation, y_validation),
              shuffle=True, callbacks=[es, mc])

model.save(r'/content/drive/MyDrive/DataProject/model_cnn', save_format='h5')
pd.DataFrame(history.history).to_csv('/content/drive/MyDrive/DataProject/history_cnn.csv')


Epoch 1/2000
1407/1407 [==============================] - ETA: 0s - loss: 2.1226 - accuracy: 0.2983
Epoch 1: val_accuracy improved from -inf to 0.39673, saving model to best_model.h5
1407/1407 [==============================] - 253s 179ms/step - loss: 2.1226 - accuracy: 0.2983 - val_loss: 1.8039 - val_accuracy: 0.3967
Epoch 2/2000
1407/1407 [==============================] - ETA: 0s - loss: 1.7997 - accuracy: 0.4031
Epoch 2: val_accuracy improved from 0.39673 to 0.44913, saving model to best_model.h5
1407/1407 [==============================] - 246s 175ms/step - loss: 1.7997 - accuracy: 0.4031 - val_loss: 1.6673 - val_accuracy: 0.4491
Epoch 3/2000
1407/1407 [==============================] - ETA: 0s - loss: 1.6708 - accuracy: 0.4509
Epoch 3: val_accuracy improved from 0.44913 to 0.49213, saving model to best_model.h5
1407/1407 [==============================] - 238s 169ms/step - loss: 1.6708 - accuracy: 0.4509 - val_loss: 1.5474 - val_accuracy: 0.4921
Epoch 4/2000
1406/1407 [==========

In [1]:
model.save(r'/content/drive/MyDrive/DataProject/model_cnn', save_format='h5')
pd.DataFrame(history.history).to_csv('/content/drive/MyDrive/DataProject/history_cnn.csv')

NameError: ignored

# 6. Transfer learning


In [9]:
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50


In [7]:
def build_vgg16(name, weights='imagenet', optimizer='adam'):
    
    # load model
    model = VGG16(weights=weights, include_top=False, input_shape=(32,32,3))
    
    # freeze all layer
    for layer in model.layers:
        layer.trainable = False
    
    # add new classifier head
    x = GlobalAveragePooling2D()(model.output)
    x = Dense(512, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(15, activation='softmax')(x)

    # instantiate new model
    myModel = keras.Model(inputs=model.input, outputs=predictions, name=name)

    # unfreeze selected layer
    #for layer in myModel.layers[unfreeze_from:]:
    #    layer.trainable = True
          
    # compile model
    myModel.compile(
          loss='categorical_crossentropy',
          optimizer=opt, 
          metrics=['accuracy']
    )
          
    # print parameters
    myModel.summary()
    
    return myModel

def build_resnet50(name, weights='imagenet', optimizer='adam'):
    
    # load model
    model = ResNet50(weights=weights, include_top=False, input_shape=(32,32,3))
    
    # freeze all layer
    for layer in model.layers:
        layer.trainable = False
    
    # add new classifier head
    x = GlobalAveragePooling2D()(model.output)
    x = Dense(512, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(15, activation='softmax')(x)

    # instantiate new model
    myModel = keras.Model(inputs=model.input, outputs=predictions, name=name)

    # unfreeze selected layer
    #for layer in myModel.layers[unfreeze_from:]:
    #    layer.trainable = True
          
    # compile model
    myModel.compile(
          loss='categorical_crossentropy',
          optimizer=opt, 
          metrics=['accuracy']
    )
          
    # print parameters
    myModel.summary()
    
    return myModel

def build_model(name, weights='imagenet', cut_at=-1, unfreeze_from=0, optimizer='adam'):
    
    # load model
    model = VGG16(weights=weights, include_top=False, input_shape=(32,32,3))
    
    # freeze all layer
    for layer in model.layers:
        layer.trainable = False
    
    # select layer output
    if cut_at==-1:
        x = model.output
    else:
        x = model.layers[cut_at].output
        
    # add new classifier head
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(15, activation='softmax')(x)

    # instantiate new model
    myModel = keras.Model(inputs=model.input, outputs=predictions, name=name)

    # unfreeze selected layer
    #for layer in myModel.layers[unfreeze_from:]:
    #    layer.trainable = True
          
    # compile model
    myModel.compile(
          loss='categorical_crossentropy',
          optimizer=opt, 
          metrics=['accuracy']
    )
          
    # print parameters
    myModel.summary()
    
    return myModel


In [ ]:
#model = build_model('full_scratch', weights=None)
# , unfreeze_from=19
model = build_vgg16('full_scratch', weights='imagenet', optimizer=opt)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_validation, y_validation),
              shuffle=True, callbacks=[es, mc])

model.save(r'/content/drive/MyDrive/DataProject/model_vgg_imagenet', save_format='h5')
pd.DataFrame(history.history).to_csv('/content/drive/MyDrive/DataProject/history_vgg_imagenet.csv')


58900480/58889256 [==============================] - 0s 0us/step
Model: "full_scratch"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                       

In [18]:
#model = build_model('full_scratch', weights=None)
model = build_resnet50('full_scratch', weights='imagenet', optimizer='adam')

history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_validation, y_validation),
              shuffle=True)

model.save(r'/content/drive/MyDrive/DataProject/model_resnet50_imagenet', save_format='h5')
pd.DataFrame(history.history).to_csv('/content/drive/MyDrive/DataProject/history_resnet50_imagenet.csv')


Model: "full_scratch"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_6[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                       